# Módulo 11: Mapeo

## Descripción general

El mapeo de lecturas cortas con respecto a un genoma de referencia suele ser el primer paso para analizar los datos de secuenciación de nueva generación, y debe ser lo más preciso posible. Debido al elevado número de lecturas que hay que manejar, se han desarrollado numerosos algoritmos sofisticados para abordar este problema y en la actualidad existen muchas herramientas de mapeo.

### Selección de una secuencia de referencia para el mapeo

Se designa un único genoma de referencia que represente a una especie para el análisis comparativo. Un genoma de referencia completo debe tener una anotación de alta calidad y reunir el máximo nivel de apoyo experimental para la anotación estructural y funcional.

>**Nota**: Es importante tener en cuenta que, aunque un único genoma de referencia arbitrario es un enfoque frecuentemente utilizado en genómica microbiana, la elección de una referencia puede representar una fuente de errores que pueden afectar a los análisis posteriores, como la detección de polimorfismos de nucleótido único (SNP) y la inferencia filogenética.

*Lecturas adicionales*: 

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3375638/

https://academic.oup.com/nar/article/42/D1/D553/1066302 

https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1008678 

### Instalar condacolab

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

### Instalar programas

In [ ]:
# Instalar bwa
!conda install -c bioconda bwa

In [ ]:
# Install samtools
!conda config --add channels bioconda
!conda config --add channels conda-forge
!conda install -c bioconda samtools

### Descargar datos

In [ ]:
!wget https://zenodo.org/records/13750987/files/Module_11.tar.gz

### Extraer el archivo .tar.gz

In [ ]:
!tar xvf Module_11.tar.gz

## Parte 1: Evaluar archivos

Navegue hasta esta carpeta del módulo y explore su contenido. Debería ver los archivos de lectura de extremos emparejados FASTQ comprimidos:

- ERR2667737_1.fastq.gz

- ERR2667737_2.fastq.gz

En este módulo analizaremos las secuencias que  corresponden al run accession [ERR2667737](https://www.ebi.ac.uk/ena/browser/view/ERR2667737) del proyecto [PRJEB3084](https://www.ebi.ac.uk/ena/browser/view/PRJEB3084).	

Algunos datos importantes de la muestra:

- País de origen: Argentina
- Organismo: *Streptococcus pneumoniae*
- Instrumento: ILLUMINA
- Modelo del instrumento: Illumina HiSeq X Ten  
- Conteo de reads: 2148156
- Conteo de bases: 324371556 
- Nombre del centro: Wellcome Sanger Institute;SC 
- Diseño de librerías:  PAIRED
- Estrategia de librerías: WGS

Y también dos archivos FASTA de secuencias de referencia:

- Reference_sequence_GPSC1.fa

- Reference_sequence_GPSC33.fa

Alinearemos las lecturas de extremos emparejados con las dos secuencias de referencia. Las secuencias de referencia representan los linajes GPSC1 y GPSC33 de *Streptococcus pneumoniae*, y utilizaremos los resultados de la alineación para determinar a qué linaje pertenecen las lecturas.

Más información: https://pubmed.ncbi.nlm.nih.gov/31003929/

Como comprobación rápida, cuente el número de líneas en cada uno de los archivos de lectura y compruebe que tienen el mismo número. Como se trata de lecturas de extremos pareados, debería haber una lectura de cada par de lecturas en cada uno de los archivos y, por lo tanto, el mismo número de líneas (y, por lo tanto, de lecturas) en cada archivo.

In [ ]:
%cd Module_11
!ls

In [ ]:
!gzip -cdf ERR2667737_1.fastq.gz | wc -l

In [ ]:
!gzip -cdf ERR2667737_2.fastq.gz | wc -l

A continuación se ofrece una explicación de estos comandos:

`gzip`: Comando para comprimir o descomprimir archivos

`cdf`: Opciones donde c significa escribir el archivo en la salida estándar, manteniendo los archivos originales sin cambios; d significa descomprimir (a .fastq); y f significa forzar la sobreescritura de los archivos de salida y el uso de enlaces simbólicos. (En este caso, los archivos fastq.gz son enlaces simbólicos)

`|`: Canaliza el archivo .fastq de salida al siguiente comando

`wc`: comando para un programa llamado word count, proporcionando el flag `-l` le decimos a word count que cuente el número de líneas en un archivo.

Los archivos de lecturas pareadas siempre deben tener el mismo número de líneas/lecturas (el orden de las lecturas en cada archivo también es crítico), por lo que si sus dos archivos emparejados tienen un número diferente de líneas, algo ha ido mal (por ejemplo, el filtrado/recorte ha ido mal y ha corrompido la salida, o quizás se están utilizando archivos de muestras diferentes).

## Parte 2: Creación de un alineamiento

### Paso 1: Indexar el genoma de referencia

Primero crearemos un índice del archivo de secuencia de referencia "Reference_sequence_GPSC1.fa". Para referencias grandes, esto puede llevar un tiempo, pero una vez creado el índice puede reutilizarlo para múltiples muestras. Por ejemplo, si tiene 100 muestras de *Streptococcus pneumoniae*, sólo tendrá que crear el índice una vez y podrá utilizarlo para las 100 muestras.

Ejecute el comando en el terminal para ejecutar bwa:

In [ ]:
# Indexar el genoma de referencia
!bwa index Reference_sequence_GPSC1.fa

La explicación de este comando es la siguiente:

`bwa`: es la herramienta

`index`: indica a bwa que indexe el archivo de entrada

`Reference_sequence_GPSC1.fa`: es el archivo de entrada

Si lista `ls` el contenido del directorio, ahora debería ver los archivos de índice bwa, tendrán el prefijo "Reference_sequence_GPSC1.fa" y tendrán extensiones como .amb, .ann, .bwt, .pac y .sa.

In [ ]:
# Listar los archivos en el directorio
!ls

### Paso 2: Alineamiento

Para alinear las lecturas con la secuencia de referencia, escriba este comando:

>Nota: Este proceso puede tardar varios minutos

In [ ]:
# Alinear las secuencias contra el genoma de referencia
!bwa mem Reference_sequence_GPSC1.fa ERR2667737_1.fastq.gz ERR2667737_2.fastq.gz > GPSC1bwa.sam

La explicación de este comando es la siguiente:

`bwa`: es la herramienta

`mem`: indica a bwa que utilice el algoritmo mem para alinear los archivos de lectura con la referencia

`ERR2667737_1.fastq.gz`: archivo de entrada de lecturas forward

`ERR2667737_1.fastq.gz`: archivo de entrada de lecturas inversas

`Reference_sequence_GPSC1.fa`: es el archivo de entrada

`> GPSC1bwa.sam`: dirige `>` los resultados de la alineación al fichero GPSC1bwa.sam

Cuando bwa se ejecute, imprimirá mensajes en la pantalla de su terminal:

Cuando bwa haya terminado, comprueba que el fichero SAM ha sido creado usando `ls`

Ahora debería haber un archivo "GPSC1bwa.sam" en el directorio. Normalmente, un archivo SAM contiene una sola línea para cada lectura del conjunto de datos, y esta línea almacena los resultados de la alineación de cada lectura (nombre de referencia, ubicación de la alineación, cadena CIGAR, la propia secuencia de lectura, calidad, etc.).

Los archivos SAM están en formato de texto. Para visualizarlos, escriba:

In [ ]:
# Listar los archivos en el directorio

In [ ]:
# Ver las primeras líneas del archivo SAM
!head GPSC1bwa.sam

Obtendrá el siguiente resultado:

![bwa](images/bwa.jpg)

### Paso 3: Convertir el archivo SAM a BAM

Es una buena práctica convertir los archivos SAM en archivos BAM (Mapa de Alineamiento Binario), que son versiones binarias comprimidas de los mismos datos, y pueden ser ordenados e indexados fácilmente para hacer las búsquedas más rápidas. Utilizaremos samtools para convertir nuestro SAM a BAM, y ordenar e indexar el archivo BAM.

Para convertir un archivo sam en un archivo bam, escriba este comando en el terminal:

In [ ]:
# Convertir el archivo SAM a BAM
!samtools sort GPSC1bwa.sam -o GPSC1bwa.bam

La explicación de este comando es la siguiente:

`samtools`: es la herramienta

`sort`: indica a samtools que ordene el archivo SAM (GPSC1bwa.sam)

`GPSC1bwa.sam`: archivo de entrada

`-o GPSC1bwa.bam`: flag para el archivo de salida llamado GPSC1bwa.bam que son los datos ordenados en formato BAM

### Paso 4: Indexar el archivo BAM

la indexación, que se basa en datos ordenados, permite realizar búsquedas más rápidas. Escriba este comando en el terminal:

In [ ]:
# Indexar el archivo BAM
!samtools index GPSC1bwa.bam

La explicación de este comando es la siguiente:

`samtools`: es la herramienta

`index`: indica a samtools que indexe el archivo de entrada (GPSC1bwa.bam)

`GPSC1bwa.bam`: archivo de entrada

Ahora debería haber dos nuevos archivos llamados GPSC1bwa.bam y GPSC1bwa.bam.bai (el archivo índice BAM) en el directorio. Ahora vamos a listar `ls -alh` el contenido del directorio para comprobar que tenemos nuestros nuevos ficheros, y también comprobar sus tamaños.

>Nota: Si su archivo SAM es 0B (es decir, 0 bytes) entonces algo salió mal con el paso de alineación bwa, así que reinicie desde allí. Si su archivo SAM está bien (es decir, >0), pero su archivo BAM es 0B (es decir, vacío), entonces algo salió mal con su conversión de SAM a BAM, así que vuelva a hacer esa sección.

Ya no necesitamos nuestro archivo SAM original (ya que ahora tenemos el archivo BAM). Así que eliminamos `rm` el archivo SAM (GPSC1bwa.sam).

### Paso 5: Evaluación de la alineación

Samtools es uno de los programas clave en los análisis de datos de secuenciación de alto rendimiento (HTS). Tiene una amplia gama de funciones y se combina fácilmente con herramientas relacionadas como vcftools (para la llamada de variantes). Una cosa que se suele comprobar es cuántas lecturas se han alineado con la referencia (mapeadas) y cuántas no (no mapeadas). Samtools puede informarnos de esto fácilmente, utilizando las banderas SAM del alineador que aprendió en la sección 2 (formatos de archivos NGS).

La segunda columna del archivo SAM contiene la bandera para la alineación de la lectura. Si el flag incluye el número 4 en su composición entonces la lectura no está mapeada, si no incluye el número 4 entonces está mapeada.


**Número de alineaciones de lectura mapeadas**

In [ ]:
# Ver las primeras líneas del archivo BAM
!samtools view -c -F4 GPSC1bwa.bam

La explicación de este comando es la siguiente

`samtools view`: para ver el archivo bwa.bam

`-c`: contar la alineación de lectura

`-F4`: omitir la alineación de lecturas que tienen el Flag 4 unmapped

**Número de lecturas no mapeadas**

Esta vez utilizamos `-f4`, que sólo incluye la alineación de lectura que tiene la bandera no mapeada 4

In [ ]:
# Ver las primeras líneas del archivo BAM
!samtools view -c -f4 GPSC1bwa.bam

Técnicamente, el comando anterior da el número de alineaciones de lectura mapeadas, no de lecturas. Una lectura puede estar mapeada igualmente bien en múltiples posiciones (una se llamará alineamiento primario, y otras alineamiento secundario [sam flag 256], o una lectura puede estar dividida en dos partes (por ejemplo, empalme) siendo una parte el alineamiento primario y las otras suplementarias [sam flag 2048].

Por lo tanto, para obtener el número real de lecturas mapeadas es necesario contar los alineamientos que no tienen las banderas 4 (no mapeado), 256 (no primario) y 2048 (suplementario) = 4+256+2048 = 2308

**Número de lecturas mapeadas**

In [ ]:
!samtools view -c -F2308 GPSC1bwa.bam

In [ ]:
!samtools view -c -F4 -F256 -F2048 GPSC1bwa.bam

**Estadísticas**

Puede generar las estadísticas de mapeo utilizando el comando:

In [ ]:
!samtools stats GPSC1bwa.bam > GPSC1bwa_bamstats.txt

Abra el archivo "GPSC1bwa_bamstats.txt" para ver las estadísticas de todas las lecturas.

## Parte 3: Visualizacion de mapeo en IGV

Descargue [IGV](https://igv.org/doc/desktop/#DownloadPage/) 

Necesitará los siguientes archivos para ver las lecturas mapeadas en IGV:

- Secuencia de referencia que utilizó para mapear sus lecturas

- Archivo BAM de lecturas mapeadas ordenadas e indexadas

Inicie IGV

1. Cargue la secuencia de referencia: En la barra de herramientas, haga clic en Genoma > Cargar genoma desde archivo > Buscar y seleccione Reference_sequence_GPSC1.fa.

2. Cargue el archivo BAM: Vaya a Archivo > Cargar desde archivo > Seleccione GPSC1bwa.bam

Ahora es libre de investigar diferentes áreas y las alineaciones en el genoma.

## Parte 4: Ahora realize un nuevo alienamiento seguiendo los pasos anteriores utilizando como genoma de referencia `Reference_sequence_GPSC33`

> Nota: Esencialmente, usted tendrá que indexar el nuevo genoma de referencia, cambiar los nombres por el nuevo genoma de referencia en el comando bwa y todos los nombres de archivo SAM/BAM en los comandos bwa y samtools de GPSC1 a GPSC33. Ejemplo:

`bwa mem Reference_sequence_GPSC33.fa ERR2667737_1.fastq.gz ERR2667737_2.fastq.gz > GPSC33bwa.sam`.

*Adaptado de:*

- Advanced Bioinformatics Course developed for the GPS and JUNO projects - Wellcome Sanger Insitute
    
*Modificado por Luisa Sacristán (Universidad de los Andes-CABANA)*